<a href="https://colab.research.google.com/github/520817/gp/blob/main/adhd(csv_%EC%B6%94%EA%B0%80%EB%A1%9C_%EB%A7%8C%EB%93%A4%EC%96%B4%EC%84%9C_%EB%AA%A8%EB%8D%B8_%EA%B5%AC%ED%98%84)_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import random
import numpy as np
import torch.backends.cudnn as cudnn

torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(42)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.cuda.is_available()

True

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
!git clone https://github.com/kmounlp/NER.git

fatal: destination path 'NER' already exists and is not an empty directory.


In [ ]:
import os
import glob

In [ ]:
file_list=[]

In [ ]:
for x in os.walk('NER/'):
  for y in glob.glob(os.path.join(x[0], '*_NER.txt')):
    file_list.append(y)

In [ ]:
file_list = sorted(file_list)

In [ ]:
from pathlib import Path

file_path = file_list[0]
file_path = Path(file_path)
raw_text = file_path.read_text().strip()

In [ ]:
!pip install korpora

from Korpora import Korpora
corpus = Korpora.load("naver_changwon_ner")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : 네이버 + 창원대
    Repository : https://github.com/naver/nlp-challenge/tree/master/missions/ner
    References : http://air.changwon.ac.kr/?page_id=10

    개체명(Named Entity)은 인명, 기관명, 지명 등과 같이 문장 또는 문서에서 특정한 의미를 가지고 있는 단어 또는 어구를 말합니다.
    이 때문에 개체명은 정보 검색 및 언어 이해를 위한 분석에서 주요한 대상으로 다루어지고 있습니다.
    Data.ly에서는 개체명 코퍼스를 제공하여 연구에 도움을 드리고자 하며, 공개적인 리더보드를 통해 많은 분들의 연구 동향을 논의/공유하고자 합니다.
    제공되는 코퍼스는 Data.ly에서 제작한 것으로, 연구 및 리더보드를 위한 학습으로 사용 가능하며 상업적인 목적으로 사용될 수 없습니다.

    # License
    연구 및 리더보드를 위한 학습으로 사용 가능하며 상업적인 목적으로 사용될 수 없습니다.

[Korpora] Corpus `naver_changwon_ner` is already installed at /root/Korpora/naver_changwon_ner/train_data


옆에 파일 눌러서 ... 누른 후 파일 date.csv 업로드 따로 해야돼요!

In [ ]:
import pandas as pd
dat = pd.read_csv("/date.csv", encoding = "CP949")

In [ ]:
dat_df = dat.iloc[:,0:1]

네이버 데이터셋 전처리

In [ ]:
def naver_read_file(file_list):


    token_docs = []
    tag_docs = []

    for doc in file_list:
        tokens = []
        tags = []
        list1=doc.words
        list2=doc.tags



        for text,docs in zip(list1,list2):
            try:
                tag = docs
                    # 2: pos, 3: ner
                if tag in ['DAT_B', 'DAT_I']:
                    if tag == 'DAT_B' or 'DAT_I':
                        token = random.sample(dat_df['date'].tolist(), k=1)[0] #tag가 B-ORG이면 상장법인목록의 회사명 중 하나로 랜덤으로 대체
                    else:
                        token = text
                else:
                    token = text

                for i, syllable in enumerate(token): # 음절 단위로 자르고
                    tokens.append(syllable)
                    if tag == '-':
                          tag = 'O' # 태그가 '-'인 경우 'O'로 변경
                    modi_tag = tag
                    if i > 0:
                        if tag[0] == 'B':
                            modi_tag = 'I' + tag[1:]     # BIO tag를 부착

                    tags.append(modi_tag)
            except:
                continue
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [ ]:
naver_text,naver_tags =naver_read_file(corpus.train)

In [ ]:
print(len(naver_text))
print(len(naver_tags))

90000
90000


In [ ]:
print(naver_text[4], end='\n\n') # 음절 단위로 잘 잘렸네요!
print(naver_tags[4])

['▲', '퍼', '거', '슨', '씨', '족', '의', '꾀']

['O', 'PER_B', 'PER_B', 'PER_B', 'CVL_B', 'CVL_B', 'CVL_B', 'O']


해양 뭐시기 데이터셋 전처리

In [ ]:
import re
def read_file(file_list):
    token_docs = []
    tag_docs = []
    for file_path in file_list:
        # print("read file from ", file_path)
        file_path = Path(file_path)
        raw_text = file_path.read_text().strip()
        raw_docs = re.split(r'\n\t?\n', raw_text)
        for doc in raw_docs:
            tokens = []
            tags = []
            for line in doc.split('\n'):
                if line[0:1] == "$" or line[0:1] == ";" or line[0:2] == "##":
                    continue
                try:
                    tag = line.split('\t')[3]# 2: pos, 3: ner

                    if tag in ['B-DAT', 'I-DAT']:
                        if tag == 'B-DAT':
                            token = random.sample(dat_df['date'].tolist(), k=1)[0]
                        elif tag == 'I-DAT':
                            token = None
                        else:
                            token = line.split('\t')[0]
                    else:
                        token = line.split('\t')[0]

                    for i, syllable in enumerate(token):    # 음절 단위로 잘라서
                        tokens.append(syllable)
                        modi_tag = tag
                        if i > 0:
                            if tag[0] == 'B':
                                modi_tag = 'I' + tag[1:]    # BIO tag를 부착할게요 :-)
                        tags.append(modi_tag)
                except:
                    continue
            token_docs.append(tokens)
            tag_docs.append(tags)

    return token_docs, tag_docs

In [ ]:
texts, tags = read_file(file_list[:])

In [ ]:
print(len(texts))
print(len(tags))

19263
19263


In [ ]:
print(texts[5], end='\n\n') # 음절 단위로 잘 잘렸네요!
print(tags[5])

['첫', '_', '회', '를', '_', '시', '작', '으', '로', '_', '6', '0', '~', '7', '0', '년', '도', '까', '지', '_', '4', '일', '간', '_', '총', '_', '4', '회', '에', '_', '걸', '쳐', '_', '매', '_', '회', '_', '2', '편', '씩', '_', '총', '_', '8', '편', '이', '_', '공', '개', '될', '_', '예', '정', '이', '다', '.']

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DAT', 'I-DAT', 'I-DAT', 'I-DAT', 'I-DAT', 'I-DAT', 'I-DAT', 'O', 'O', 'O', 'B-DUR', 'I-DUR', 'I-DUR', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'B-NOH', 'I-NOH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


분리된 토큰을 하나의 개체명으로 묶어 주는 태깅 시스템을 위해 있는게 BIO, BIESO임.

여러 개의 토큰으로 이루어진 개체명의 경우

**개체명이 시작할 때** 'B,begin'

**토큰이 개체명 중간에 있을 때** 'I,inside'

**개체명의 마지막에 위치할 때** 'E,end'

**하나의 토큰이 곧 하나의 개체명일 때** 'S,singleton'

**토큰이 개체명이 아닐 경우** 'O,outside'

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

for i, tag in enumerate(unique_tags):
    print(tag)  #해양 tags 변경 전

I-DAT
B-PNT
I-PNT
I-ORG
B-ORG
B-TIM
I-DUR
B-DUR
B-DAT
B-NOH
I-POH
B-POH
O
I-MNY
B-LOC
I-PER
I-LOC
I-TIM
B-MNY
B-PER
I-NOH


In [ ]:
for sublist in tags:
    for i, label in enumerate(sublist):
        if label == 'I-NOH':
            sublist[i] = 'NOH_I'
        elif label == 'I-PNT':
            sublist[i] = 'PNT_I'
        elif label == 'I-ORG':
            sublist[i] = 'ORG_I'
        elif label == 'I-DUR':
            sublist[i] = 'DUR_I'
        elif label == 'I-TIM':
            sublist[i] = 'TIM_I'
        elif label == 'I-PER':
            sublist[i] = 'PER_I'
        elif label == 'I-MNY':
            sublist[i] = 'MNY_I'
        elif label == 'B-MNY':
            sublist[i] = 'MNY_B'
        elif label == 'B-DAY':
            sublist[i] = 'DAY_B'
        elif label == 'B-TIM':
            sublist[i] = 'TIM_B'
        elif label == 'I-LOC':
            sublist[i] = 'LOC_I'
        elif label == 'B-PER':
            sublist[i] = 'PER_B'
        elif label == 'B-ORG':
            sublist[i] = 'ORG_B'
        elif label == 'B-LOC':
            sublist[i] = 'LOC_B'
        elif label == 'B-PNT':
            sublist[i] = 'PNT_B'
        elif label == 'B-NOH':
            sublist[i] = 'NOH_B'
        elif label == 'I-PER':
            sublist[i] = 'PER_I'
        elif label == 'B-LOC':
            sublist[i] = 'LOC_B'
        elif label == 'I-POH':
            sublist[i] = 'POH_I'
        elif label == 'I-DAT':
            sublist[i] = 'DAT_I'
        elif label == 'B-POH':
            sublist[i] = 'POH_B'
        elif label == 'B-DUR':
            sublist[i] = 'DUR_B'
        elif label == 'B-DAT':
            sublist[i] = 'DAT_B'
        elif label == '0':
            sublist[i] = '0'

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

for i, tag in enumerate(unique_tags):
    print(tag)  #해양 tags 변경 후

PER_B
TIM_I
POH_I
DUR_B
TIM_B
POH_B
LOC_B
O
PNT_B
NOH_B
MNY_B
PNT_I
LOC_I
ORG_I
DAT_I
DUR_I
NOH_I
ORG_B
DAT_B
PER_I
MNY_I


In [ ]:
texts.extend(naver_text)
tags.extend(naver_tags)

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
for i, tag in enumerate(unique_tags):
    print(tag)  # 학습을 위한 label list를 확인합니다.

PER_B
TRM_B
TIM_I
POH_I
AFW_I
ANM_I
PLT_I
DUR_B
FLD_B
TIM_B
POH_B
EVT_B
TRM_I
ANM_B
LOC_B
EVT_I
FLD_I
O
NUM_I
AFW_B
PNT_B
NOH_B
MNY_B
MAT_I
CVL_I
PNT_I
LOC_I
ORG_I
DAT_I
DUR_I
NOH_I
PLT_B
CVL_B
NUM_B
MAT_B
ORG_B
DAT_B
PER_I
MNY_I


In [ ]:
for tag in list(tag2id.keys()) :
    globals()[tag] = 0

for tag in tags :
    for ner in tag :
        globals()[ner] += 1

for tag in list(tag2id.keys()) :
    print('{:>6} : {:>7,}'. format(tag, globals()[tag]))

 PER_B : 192,859
 TRM_B : 129,695
 TIM_I :   5,511
 POH_I :  37,156
 AFW_I :   9,433
 ANM_I :     172
 PLT_I :       5
 DUR_B :   1,207
 FLD_B :   9,715
 TIM_B :  10,831
 POH_B :   6,686
 EVT_B :  59,714
 TRM_I :  14,924
 ANM_B :  22,070
 LOC_B :  98,439
 EVT_I :  31,340
 FLD_I :     155
     O : 3,067,701
 NUM_I :  26,913
 AFW_B :  23,535
 PNT_B :   1,672
 NOH_B :  11,051
 MNY_B :   1,440
 MAT_I :      62
 CVL_I :  15,293
 PNT_I :   4,613
 LOC_I :  17,533
 ORG_I :  68,227
 DAT_I :  66,187
 DUR_I :   4,573
 NOH_I :  23,967
 PLT_B :     952
 CVL_B : 266,517
 NUM_B : 228,990
 MAT_B :   1,001
 ORG_B : 255,988
 DAT_B : 143,060
 PER_I :  52,397
 MNY_I :   6,930


In [ ]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_tags, test_tags = train_test_split(texts, tags, test_size=.2, random_state=42)

In [ ]:
print('Train 문장 : {:>6,}' .format(len(train_texts)))
print('Train 태그 : {:>6,}' .format(len(train_tags)))
print('Test  문장 : {:>6,}' .format(len(test_texts)))
print('Test  태그 : {:>6,}' .format(len(test_tags)))

Train 문장 : 87,410
Train 태그 : 87,410
Test  문장 : 21,853
Test  태그 : 21,853


In [ ]:
train_texts[5]

['첫',
 '출',
 '발',
 '남',
 '상',
 '은',
 '작',
 '은',
 '어',
 '간',
 '하',
 '나',
 '였',
 '어',
 '요',
 '.']

BERT 토크나이저

In [ ]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
pad_token_id = tokenizer.pad_token_id # 0
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

In [ ]:
def ner_tokenizer(sent, max_seq_length):
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 우리가 구성한 학습 데이터도 이렇게 구성되었기 때문이라고 함.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1

    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [ ]:
print(ner_tokenizer(train_texts[0], 5))

{'input_ids': [2, 1912, 4044, 4040, 3], 'attention_mask': [1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0]}


In [ ]:
tokenized_train_sentences = []
tokenized_test_sentences = []

for text in train_texts:    # 전체 데이터를 tokenizing 합니다.
    tokenized_train_sentences.append(ner_tokenizer(text, 128))
for text in test_texts:
    tokenized_test_sentences.append(ner_tokenizer(text, 128))

In [ ]:
def encode_tags(tags, max_seq_length):
    # label 역시 입력 token과 개수를 맞춰줍니다
    tags = tags[:max_seq_length-2]
    labels = [tag2id[tag] for tag in tags]
    labels = [tag2id['O']] + labels

    padding_length = max_seq_length - len(labels)
    labels = labels + ([pad_token_label_id] * padding_length)

    return labels

In [ ]:
tag2id

{'PER_B': 0,
 'TRM_B': 1,
 'TIM_I': 2,
 'POH_I': 3,
 'AFW_I': 4,
 'ANM_I': 5,
 'PLT_I': 6,
 'DUR_B': 7,
 'FLD_B': 8,
 'TIM_B': 9,
 'POH_B': 10,
 'EVT_B': 11,
 'TRM_I': 12,
 'ANM_B': 13,
 'LOC_B': 14,
 'EVT_I': 15,
 'FLD_I': 16,
 'O': 17,
 'NUM_I': 18,
 'AFW_B': 19,
 'PNT_B': 20,
 'NOH_B': 21,
 'MNY_B': 22,
 'MAT_I': 23,
 'CVL_I': 24,
 'PNT_I': 25,
 'LOC_I': 26,
 'ORG_I': 27,
 'DAT_I': 28,
 'DUR_I': 29,
 'NOH_I': 30,
 'PLT_B': 31,
 'CVL_B': 32,
 'NUM_B': 33,
 'MAT_B': 34,
 'ORG_B': 35,
 'DAT_B': 36,
 'PER_I': 37,
 'MNY_I': 38}

In [ ]:
encode_tags(train_tags[0], 5)

[17, 0, 0, 0, 17]

In [ ]:
train_labels = []
test_labels = []

for tag in train_tags:
    train_labels.append(encode_tags(tag, 128))

for tag in test_tags:
    test_labels.append(encode_tags(tag, 128))

In [ ]:
len(train_labels), len(test_labels)

(87410, 21853)

Token 데이터셋

In [ ]:
import torch

class TokenDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.encodings[idx].items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TokenDataset(tokenized_train_sentences, train_labels)
test_dataset = TokenDataset(tokenized_test_sentences, test_labels)

In [ ]:
#!pip install --upgrade pip setuptools wheel
#!pip install —-upgrade tokenizers
#!pip install —-upgrade Cython
#!pip install accelerate -U
#!pip install transformers==4.3.0

In [ ]:
#!pip install --upgrade transformers

In [ ]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments, AutoModelForTokenClassification,EarlyStoppingCallback
import sys
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000, # 1000번쨰 steps마다 log를 보여줌
    learning_rate=3e-5,
    weight_decay=0.01,
    save_total_limit=5,
    save_strategy='steps', # steps로 해야 earlystop이 가능
    evaluation_strategy='steps',
    save_steps=1000, # 1000번쨰 step마다 저장
    eval_steps=1000, # 1000번째 step마다 평가
    seed=15,
    load_best_model_at_end=True # 가장 좋은 성능의 모델로...
)

BertForTokenClassification

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(unique_tags))
model.to(device)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,# evaluation dataset
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)] #loss가 2번 감소하지 않으면 스탑
)

pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import gc
gc.collect()

107

In [ ]:
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
trainer.evaluate()

In [ ]:
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
index_to_ner = {i:j for j, i in tag2id.items()}
f_label = [i for i, j in tag2id.items()]
val_tags_l = [index_to_ner[x] for x in np.ravel(predictions.label_ids).astype(int).tolist()]
y_predicted_l = [index_to_ner[x] for x in np.ravel(preds).astype(int).tolist()]

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
#데이터셋 추가
print(classification_report(val_tags_l, y_predicted_l, labels=f_label))

In [ ]:
trainer.save_model('kcelectra_base_new')

모델이 저장되면 옆에 파일 누르면 content 파일에 모델이 저장될 거예요. 커서를 우측으로 갖다대면 점 세개가 나오는데 거기서 다운로드를 받아야해요. config든 뭐든 다 다운로드 받아야하고 시간 걸릴 거예요.

In [ ]:
len(tag2id.values())

In [ ]:
# 저장한 모델 불러오기
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tag2id = {'POH_B': 0, 'CVL_B': 1, 'FLD_B': 2, 'FLD_I': 3, 'EVT_I': 4,'AFW_B': 5,'CVL_I': 6,'NUM_B': 7,'MAT_I': 8, 'TIM_I': 9,'ANM_I': 10,'PER_I': 11,'POH_I': 12,
 'TRM_I': 13,
 'TIM_B': 14,
 'ANM_B': 15,
 'O': 16,
 'DAT_I': 17,
 'DUR_I': 18,
 'PNT_B': 19,
 'PNT_I': 20,
 'MNY_I': 21,
 'EVT_B': 22,
 'ORG_I': 23,
 'MNY_B': 24,
 'ORG_B': 25,
 'LOC_B': 26,
 'PLT_I': 27,
 'MAT_B': 28,
 'DAT_B': 29,
 'NUM_I': 30,
 'AFW_I': 31,
 'PLT_B': 32,
 'DUR_B': 33,
 'PER_B': 34,
 'LOC_I': 35,
 'NOH_B': 36,
 'TRM_B': 37,
 'NOH_I': 38}
unique_tags={'POH_B', 'CVL_B', 'FLD_B', 'FLD_I', 'EVT_I', 'AFW_B', 'CVL_I', 'NUM_B', 'MAT_I', 'TIM_I', 'ANM_I', 'PER_I', 'POH_I', 'TRM_I', 'TIM_B', 'ANM_B', 'O', 'DAT_I', 'DUR_I', 'PNT_B', 'PNT_I', 'MNY_I', 'EVT_B', 'ORG_I', 'MNY_B', 'ORG_B', 'LOC_B', 'PLT_I', 'MAT_B', 'DAT_B', 'NUM_I', 'AFW_I', 'PLT_B', 'DUR_B', 'PER_B', 'LOC_I', 'NOH_B', 'TRM_B', 'NOH_I'}
id2tag={0: 'POH_B',
 1: 'CVL_B',
 2: 'FLD_B',
 3: 'FLD_I',
 4: 'EVT_I',
 5: 'AFW_B',
 6: 'CVL_I',
 7: 'NUM_B',
 8: 'MAT_I',
 9: 'TIM_I',
 10: 'ANM_I',
 11: 'PER_I',
 12: 'POH_I',
 13: 'TRM_I',
 14: 'TIM_B',
 15: 'ANM_B',
 16: 'O',
 17: 'DAT_I',
 18: 'DUR_I',
 19: 'PNT_B',
 20: 'PNT_I',
 21: 'MNY_I',
 22: 'EVT_B',
 23: 'ORG_I',
 24: 'MNY_B',
 25: 'ORG_B',
 26: 'LOC_B',
 27: 'PLT_I',
 28: 'MAT_B',
 29: 'DAT_B',
 30: 'NUM_I',
 31: 'AFW_I',
 32: 'PLT_B',
 33: 'DUR_B',
 34: 'PER_B',
 35: 'LOC_I',
 36: 'NOH_B',
 37: 'TRM_B',
 38: 'NOH_I'}
pad_token_id = tokenizer.pad_token_id # 0
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

model = AutoModelForTokenClassification.from_pretrained('kcelectra_base_new', num_labels=len(unique_tags))
model.to(device)

In [ ]:
# 기존 토크나이저는 wordPiece tokenizer로 tokenizing 결과를 반환합니다.
# 데이터 단위를 음절 단위로 변경했기 때문에, tokenizer도 음절 tokenizer로 변경

# berttokenizer를 사용하는데 한국어 vocab이 8000개 정도 밖에 없고 그 안의 한국어들의 거의 음절로 존재
# -> 음절 단위 tokenizer를 적용하면 vocab id를 어느 정도 획득할 수 있어 UNK가 별로 없을듯 하다
def ner_tokenizer(sent, max_seq_length):
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 우리가 구성한 학습 데이터도 이렇게 구성되었기 때문이라고 함.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1

    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [ ]:
def ner_inference(text) :

    model.eval()
    text = text.replace(' ', '_')

    predictions , true_labels = [], []

    tokenized_sent = ner_tokenizer(text, len(text)+2)
    input_ids = torch.tensor(tokenized_sent['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(tokenized_sent['attention_mask']).unsqueeze(0).to(device)
    token_type_ids = torch.tensor(tokenized_sent['token_type_ids']).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

    logits = outputs['logits']
    logits = logits.detach().cpu().numpy()
    label_ids = token_type_ids.cpu().numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.append(label_ids)

    pred_tags = [list(tag2id.keys())[p_i] for p in predictions for p_i in p]

    print('{}\t{}'.format("TOKEN", "TAG"))
    print("===========")
    # for token, tag in zip(tokenizer.decode(tokenized_sent['input_ids']), pred_tags):
    #   print("{:^5}\t{:^5}".format(token, tag))
    for i, tag in enumerate(pred_tags):
        print("{:^5}\t{:^5}".format(tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'][i]), tag))

In [ ]:
text='모레 10시 반 안과 방문 예정, 시력 물어보기'

In [ ]:
ner_inference(text)